# SD201 Project 

## Dataset (from a Kaggle competition) : Instacart Market Basket Analysis

Link : https://www.kaggle.com/c/instacart-market-basket-analysis/data

Blog post about the competition : https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2

Key points from the dataset:

- 3M grocery store orders
- 200,000+ Instacart users
- 4 to 100 orders for each user, timestamped

“The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.instacart.com/datasets/grocery-shopping-2017 on 10/12/2021"

## Introduction

In this notebook, we perform compare and decide on the best classication algorithm to apply on the Instacart data set.

## Setup (run all) 

In [1]:
# # Run cell if using Google Colab
# # Mount the private Google Drive folder to access the .csv files
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

In [2]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [3]:
'''Python librairies''' 

# Utility librairies
import pandas as pd
import scipy.stats as s
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

# Preprocessing and pipeline librairies
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

# Wrapper to convert regular classifiers to multi-label classifiers
from sklearn.multioutput import MultiOutputClassifier

# Classifiers that support multi-label output
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
import xgboost as xgb

# Kernel approximation for SVM methods (unused)
from sklearn.kernel_approximation import Nystroem

# Metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import hamming_loss, jaccard_score

# Pretty charts
import seaborn as sns
sns.set_theme(style="ticks")

In [4]:
# Open the data
path_to_csv = './instacart/'

op_prior = pd.read_csv(path_to_csv + 'order_products__prior.csv')
op_train = pd.read_csv(path_to_csv + 'order_products__train.csv')
orders   = pd.read_csv(path_to_csv + 'orders.csv')
products = pd.read_csv(path_to_csv + 'products.csv')

## Data cleaning

The products DataFrame has some missing items (the indexes skip at some points and become slightly offset). This may lead to issues later on when using loc (from pandas) to locate data. We choose to set the index as the product_id.

In [5]:
products.set_index('product_id', inplace=True)

We can check that the only problem we have with missing values are in the `orders` file, in which a certain number of `days_since_prior_order` entries are missing.

In [6]:
orders.isnull().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [7]:
np.sort(orders.days_since_prior_order.unique())

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., nan])

We need to figure out what to do with `nan` entries in days_since_prior order.

What exactly are those entries? Counting the number of unique clients, we get that there are exactly as many null entries as there are clients.

In [8]:
len(orders.user_id.unique())

206209

Intuitively, we can guess that those null entries correspond to the first order for each of those clients. We can check this by counting the number of null entries for each client.

In [9]:
orders.days_since_prior_order.isnull().groupby(orders['user_id']).sum().unique()

array([1], dtype=int64)

We indeed get exactly one `nan` entry for each client, which confirms our assumption that those entries correpond to the first order for each client.

What could be a good strategy for dealing with those entries?

We could try either to drop them entirely from our dataset, or to backfill them with some value (be it the average elasped time for all orders, or the average elapsed time between two orders for the given client).

Since most clients only have 4 orders, dropping the first order would mean dropping 25% of the data for this client. For this reason, we choose not to drop any row.

We can compare both methods using pipelines.

### Formatting the data 

We cannot exploit our relational data directly: we need to perform merges using the keys in the data, and then perform an aggregation over the ordered products to get arrays for each cart.

Moreover, we do not need to perform a merge over the `products` DataFrame because the data there is redundant and already fully determined by the `product_id`. Therefore performing a merge there would induce data-leakage when training our model.

In [10]:
threshold = 10e-4
order_count = len(op_prior)

# Create the DataFrame of ordered products with their frequencies
item_freq = op_prior.product_id.value_counts(ascending=False)
item_freq = pd.DataFrame(item_freq.reset_index())
item_freq.rename(columns={'product_id':'n_occ', 'index':'product_id'}, inplace= True)
item_freq['frequency'] = item_freq['n_occ']/order_count

In [11]:
# Compare the number of products before and after the drop
bf_size = len(item_freq)
item_freq = item_freq[item_freq.frequency>threshold]
af_size = len(item_freq)
print('Number of products before :', bf_size, 'after:', af_size)

Number of products before : 49677 after: 101


In [12]:
# # Keep first item in cart
# first_item_prior = op_prior[op_prior.add_to_cart_order == 1]

# Drop all rows with unfrequently bought products
op_prior = op_prior[op_prior.product_id.isin(item_freq.product_id)]

In [13]:
def arrange_data(op_data):
    '''
    Format the data so that to each order corresponds an array of product_id (the cart),
    and an array indicating whether an item was reordered or not.
    op_data can be either op_train or op_prior.
    '''
    data = orders.merge(op_data[['order_id', 'product_id']], on='order_id')
    
    # Aggregate the carts into arrays
    groupby_cols = ['order_id',
                'user_id',
                'eval_set',
                'order_number',
                'order_dow',
                'order_hour_of_day',
                'days_since_prior_order']
    
    data = data.groupby(groupby_cols).aggregate(list)
    
    # Rename the product_id column to 'cart'
    data.rename(columns = {'product_id':'cart'}, inplace = True)
    
    # Reset the index that was changed by the aggregation
    data = data.reset_index()
    
    return data

In [14]:
# Create the DataFrame with aggregated carts for each order
train_data = arrange_data(op_prior)

In [15]:
# Free the RAM for Google Colab
op_prior = None

In [16]:
# # Change the data types for all columns to optimize memory use.

# # We compare the memory saved by changing the data types.
# before_m = train_data.memory_usage().sum()

# # We must signal to Python that these columns represent categorical data 
# # so that the models work correctly.
# train_data['order_id'] = train_data['order_id'].astype('category')
# train_data['user_id'] = train_data['user_id'].astype('category')
# train_data['eval_set'] = train_data['eval_set'].astype('category')
# train_data['order_number'] = train_data['order_number'].astype('category')

# train_data['order_dow'] = train_data['order_dow'].astype('uint8')
# train_data['order_hour_of_day'] = train_data['order_hour_of_day'].astype('uint8')
# train_data['days_since_prior_order'] = train_data['days_since_prior_order'].astype('float16')
# after_m = train_data.memory_usage().sum()

# # How much memory have we saved ?
# (before_m-after_m)/before_m

## Feature engineering (unused)

Section is discarded for this project

#### Creating new features 

In [17]:
# # Number of items in the cart
# train_data['n_products'] = train_data.cart.apply(lambda x: len(x))
# train_data['n_products'] = train_data['n_products'].astype('uint8')

In [18]:
# # Number of reordered items in the cart
# train_data['n_reord'] = train_data.reordered.apply(lambda x: np.count_nonzero(x))
# train_data['n_reord'] = train_data['n_reord'].astype('uint8')

In [19]:
# # Average number of days elapsed between orders for a given client
# train_data.groupby('user_id').days_since_prior_order.mean()

Ideas:

- number of reordered products
- fraction of reordered products
- probability of a product to be reordered
- convert date

## Data mining 

### Multi-label classification based on time of order

#### Models and pipelines definition

Our goal here is to choose the best classification algorithm for the problem at hand, based on the fitting time and a metric score.

To effectively apply algorithms for a multi-label problem, we either need to transform our carts into a sparse matrix using `sklearn.preprocessing.MultiLabelBinarizer`, or to use `sklearn.multioutput.MultiOutputClassifier` to extend a single variable classifier into a multi-label one.

In [20]:
numerical_cols = ['order_dow', 'order_hour_of_day', 'days_since_prior_order']
# Impute the average over all orders
avg_imp = SimpleImputer(missing_values=np.nan, strategy='mean')

# Imputing the average for a given client in a pipeline necessitates writing a custom imputer.
# This is optional and will be done if there is enough time.

# Min-max normalization 
mm_scaler = MinMaxScaler()
std_scaler = StandardScaler()

# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', avg_imp, numerical_cols),
        ('norm', std_scaler, numerical_cols)
    ])

In [21]:
# Create the different models

# Random Forest Classifier
RF_model = RandomForestClassifier(n_estimators=100, verbose=True)
multi_RF_model = MultiOutputClassifier(RF_model, n_jobs=-1)

# Linear SVC Classifier
# Set dual to false as samples>>features
LSVC_model = LinearSVC(dual=False, verbose=True)
multi_LSVC_model = MultiOutputClassifier(LSVC_model, n_jobs=-1)

# SGD Classifier
# Use Nystroem transform to get an approximate kernel
SGD_model = SGDClassifier(verbose=True)
multi_SGD_model = MultiOutputClassifier(SGD_model, n_jobs=-1)

# kNN Classifier
# Use GridSearchCV to tune k
kNN_model = KNeighborsClassifier()
multi_kNN_model = MultiOutputClassifier(kNN_model, n_jobs=-1)

#MLP Classifier
MLP_model = MLPClassifier(verbose=True)
multi_MLP_model = MultiOutputClassifier(MLPClassifier(), n_jobs=-1)

# lgbm Classifier
lgbm_model = lgb.LGBMClassifier()
multi_lgbm_model = MultiOutputClassifier(lgbm_model, n_jobs=-1)

# xgb Classifier
xgb_model = xgb.XGBClassifier()
multi_xgb_model = MultiOutputClassifier(xgb_model, n_jobs=-1)

# # Default Classifier
# model = example_model()
# multi_model = MultiOutputClassifier(model, n_jobs=-1)

In [22]:
# Make the different pipelines

# Random Forest Classifier
# Not compatible with sparse matrix output
RF_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('RF model', multi_RF_model)
                             ],
                       verbose=True)

# Linear SVC Classifier
LSVC_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('LSVC model', multi_LSVC_model)
                             ],
                        verbose=True)

# SGD Classifier
SGD_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('SGD model', multi_SGD_model)
                             ],
                        verbose=True)

# kNN Classifier
kNN_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('kNN model', multi_kNN_model)
                             ],
                        verbose=True)

# MLP Classifier
MLP_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('MLP model', multi_MLP_model)
                             ],
                        verbose=True)

# lgbm Classifier
lgbm_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('lgbm model', multi_lgbm_model)
                             ],
                        verbose=True)

# lgbm Classifier
xgb_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('xgb model', multi_xgb_model)
                             ],
                        verbose=True)

# # Default Classifier
# pipeline = Pipeline(steps=[('preprocessor', preprocessor),
#                               ('OVR model', OVR_model)
#                          ],
#                     verbose=True)

In [23]:
# Train and validate the models
features = ['order_dow', 'order_hour_of_day', 'days_since_prior_order']
target = 'cart'

# Reduce dataset size to overcome training memory issues...
train_sample = train_data.sample(axis=0, frac=1/50)

# Define target and features
X = train_sample[features]
y = train_sample[target]

# Fit the MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

# Convert the target data into binary matrix
y_train_bm = mlb.transform(y_train)
y_test_bm = mlb.transform(y_test)

In [24]:
# # Apply Nystroem kernel approximation (unused)
# # Used eventually for SGD or LSVC models
# feature_map_nystroem = Nystroem(gamma=.2,
#                                 random_state=1,
#                                 n_components=300)
# X_train_nys = feature_map_nystroem.fit_transform(X_train)

#### Utility functions for fitting and comparing models

We define functions for fitting, scoring, and comparing all models at once.

In [26]:
def fit_score(model, X_train, X_test, y_train, y_test):
    """Fits the given multi-label model and scores it
    using the four metrics below"""
    # Fit the model
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)

    # Score the model
    f1 = f1_score(y_test, y_pred, average='samples')
    accuracy = accuracy_score(y_test, y_pred)
    hamming = hamming_loss(y_test, y_pred)
    jaccard = jaccard_score(y_test, y_pred, average='samples')

    return model, y_pred, f1, hamming, accuracy, jaccard
  
def compare_models(pipelines):
    """Fit and compare models based on speed and four metrics"""
    models = []
    predictions = []
    for p in pipelines:
        model, y_pred, f1, hamming, accuracy, jaccard = fit_score(p,
                                                              X_train,
                                                              X_test,
                                                              y_train_bm,
                                                              y_test_bm)
        models.append(model)
        predictions.append(y_pred)
        print('f1-score:', f1)
        print('accuracy score:', accuracy)
        print('hamming loss:', hamming)
        print('jaccard score:', jaccard)
    return models, predictions

We also define functions that make the predictions human readable.

In [31]:
def convert_to_carts(y_pred):
    """Convert back binary matrix prediction outputs to human readable carts"""
    arr = mlb.inverse_transform(y_pred)
    carts = [[] for i in range (len(arr))]
    for i in range(len(arr)):
        for id in arr[i]:
            carts[i].append(products.loc[id].product_name)
    return carts

def print_carts(y_pred):
    """Print carts and number of empty carts.
     y_ pred must be in binary matrix format."""
    carts = pd.Series(convert_to_carts(y_pred))
    empty = 0
    for cart in carts:
        if cart == []:
            empty+=1
        else:
            print(cart)
    print('Number of empty carts:', empty)

#### Model comparison

In [29]:
# Compare the different models with default parameters
pipelines = [RF_pipeline,
             LSVC_pipeline,
             SGD_pipeline,
             kNN_pipeline,
             MLP_pipeline,
             lgbm_pipeline,
             xgb_pipeline]

models, preds = compare_models(pipelines)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.1s
[Pipeline] .......... (step 2 of 2) Processing RF model, total= 1.2min
f1-score: 0.006305060378931516
accuracy score: 0.0012075471698113208
hamming loss: 0.03308275733233701
jaccard score: 0.00445075108475716
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.1s
[Pipeline] ........ (step 2 of 2) Processing LSVC model, total=   3.1s
f1-score: 0.0
accuracy score: 0.0
hamming loss: 0.03159349897253876
jaccard score: 0.0
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ......... (step 2 of 2) Processing SGD model, total=  10.8s
f1-score: 0.0024600529337102228
accuracy score: 0.0007547169811320754
hamming loss: 0.0337903979077153
jaccard score: 0.0018477728770181598
[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.0s
[Pipeline] ......... (step 2 of 2) Processing kNN model, total=   3.1s
f1-score: 0.010844868758076304
accuracy score: 0.002037735849056604

In [ ]:
# Fit LSVC and SGD using Nystroem transform (unused)
# m2, y_pred2, f12, hamming2, accuracy2, jaccard2 = fit_score(LSVC_pipeline,X_train_nys,X_test,y_train_bm,y_test_bm)
# m1, y_pred1, f11, hamming1, accuracy1, jaccard1 = fit_score(SGD_pipeline,X_train_nys,X_test,y_train_bm,y_test_bm)

#### Conclusions

Here we choose to use the Jaccard score to compare our models.

Conclusions:
- The Random Forest Classifier takes a long time to train but appears to work correctly.
- The Linear Support Vector Classification method does not work and only predicts empty carts.
- The Stochastic Gradient Descend method does not work and only predicts empty carts.
- k-NearestNeighbors is much quicker than using Random Forest, and results in better performances with less empty carts.
- The MultiLayerPercetron method does not work and only predicts empty carts.
- The lightGradientBoosted method returns only a handful of carts with only one or two product each.
- The ExtremeGradientBoosting method does not work and only predicts empty carts.

Therefore the k-NearestNeighbors method is priviledged for this situation.

The cell below prints carts according to the predicted results.

In [32]:
# Chose i to print carts depending on model
# 0 RF_pipeline
# 1 LSVC_pipeline
# 2 SGD_pipeline
# 3 kNN_pipeline
# 4 MLP_pipeline
# 5 lgbm_pipeline
# 6 xgb_pipeline

i=3
print_carts(preds[i])

['Strawberries', 'Banana', 'Organic Blueberries']
['Organic Raspberries']
['Banana']
['Organic Raspberries']
['Bag of Organic Bananas']
['Banana']
['Organic Avocado']
['Banana']
['Banana']
['Banana']
['Banana']
['Bag of Organic Bananas']
['Limes']
['Banana']
['Organic Strawberries']
['Banana']
['Banana']
['Bag of Organic Bananas']
['Bag of Organic Bananas']
['Bag of Organic Bananas', 'Organic Large Extra Fancy Fuji Apple']
['Bag of Organic Bananas']
['Organic Baby Spinach']
['Strawberries', 'Banana', 'Organic Blueberries']
['Banana']
['Banana']
['Organic Strawberries']
['Organic Strawberries']
['Bag of Organic Bananas']
['Organic Baby Spinach', "Organic D'Anjou Pears"]
['Bag of Organic Bananas']
['Bag of Organic Bananas']
['Banana']
['Organic Strawberries']
['Banana']
['Banana']
['Banana']
['Bag of Organic Bananas']
['Organic Baby Spinach']
['Banana']
['Banana']
['Bag of Organic Bananas']
['Banana']
['Organic Baby Spinach']
['Organic Baby Carrots']
['Bag of Organic Bananas']
['Banana']

['Banana']
['Bag of Organic Bananas']
['Banana']
['Banana']
['Bag of Organic Bananas']
['Banana']
['Banana']
['Large Alfresco Eggs']
['Organic Cilantro']
['Organic Baby Spinach']
['Banana']
['Organic Cilantro']
['Banana', 'Organic Avocado']
['Banana']
['Bag of Organic Bananas']
['Organic Hass Avocado']
['Bag of Organic Bananas']
['Strawberries']
['Banana']
['Bag of Organic Bananas']
['Banana']
['Banana']
['Banana']
['Banana']
['Strawberries']
['Banana']
['Banana']
['Bag of Organic Bananas']
['Banana']
['Bag of Organic Bananas']
['Organic Baby Spinach']
['100% Whole Wheat Bread']
['Large Lemon']
['Banana']
['Bag of Organic Bananas']
['Strawberries']
['Bag of Organic Bananas']
['Banana']
['Organic Hass Avocado']
['Banana']
['Banana']
['Banana']
['Banana']
['Organic Baby Spinach']
['Organic Strawberries']
['Banana']
['Organic Grape Tomatoes']
['Banana']
['Original Hummus']
['Bag of Organic Bananas']
['Banana']
['Bag of Organic Bananas', 'Organic Large Extra Fancy Fuji Apple']
['Banana']
[

['Organic Hass Avocado']
['Red Peppers', 'Raspberries']
['Banana']
['Banana', 'Organic Half & Half']
['Michigan Organic Kale', 'Organic Avocado']
['Banana']
['Banana']
['Banana']
['Organic Baby Spinach']
['Banana']
['Bag of Organic Bananas']
['Original Hummus']
['Organic Grape Tomatoes']
['Bag of Organic Bananas']
['Banana']
['Banana']
['Banana']
['Organic Italian Parsley Bunch', 'Organic Zucchini']
['Limes']
['Banana']
['Banana']
['Bag of Organic Bananas']
['Bag of Organic Bananas']
['Organic Avocado']
['Bag of Organic Bananas']
['Organic Hass Avocado']
['Organic Large Extra Fancy Fuji Apple', 'Banana']
['Organic Strawberries']
['Banana']
['Strawberries']
['Organic Strawberries']
['Organic Hass Avocado']
['Bag of Organic Bananas']
['Banana']
['Bag of Organic Bananas']
['Banana', 'Organic Half & Half']
['Organic Zucchini']
['Organic Hass Avocado']
['Organic Strawberries']
['Banana']
['Organic Reduced Fat 2% Milk']
['Organic Baby Spinach']
['Bag of Organic Bananas']
['Organic Italian Pa

We can now optimize our selected model.

## Next
See `SD201-Instacart-MultiLabel-kNN.ipynb` for the next step.